In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [2]:
def score(dice):
    """
    Returns the maximum score for a list of 1-6 dice
    Scoring follows these rules in order (k: die val, n: num of dice showing k):
    1. (k = 1, n > 2): 1000 + (n - 3) * 100
    2. (k != 1, n > 2): (k * (n - 2)) * 100
    3. (k = 1, n = 1, 2): k * 100
    4. (k = 5, n = 1, 2): k * 50
    """
    counts = [0] * 6
    for die in dice: counts[die - 1] += 1
    score = 0
    if counts[0] >= 3:  # 1
        score += 1000 + (counts[0] - 3) * 100
        counts[0] = 0
    for i in range(1, 6):  # 2
        count = counts[i]
        if count >= 3:
            score += (i + 1) * 100 + (count - 3) * 100
            counts[i] = 0
    score += counts[0] * 100  # 3
    score += counts[4] * 50  # 4
    return score

In [5]:
# Expected scores for one roll with n dice
exp_rolls = [0] * 7
for n in range(7):
    dice = [list(range(1, 7)) for _ in range(n)]
    p_roll = 6 ** -n
    exp_roll = 0
    for roll in product(*dice):
        exp_roll += score(roll) * p_roll
    print(f"{n} dice, E[Score] = {round(exp_roll, 2)}")
    exp_rolls[n] = exp_roll

0 dice, E[Score] = 0
1 dice, E[Score] = 25.0
2 dice, E[Score] = 50.0
3 dice, E[Score] = 86.81
4 dice, E[Score] = 141.67
5 dice, E[Score] = 217.07
6 dice, E[Score] = 313.05


In [6]:
# Expected score for one turn
# Naive Strategy: pull anything available, reroll only if the expected score after reroll is higher than the current score
# Equivalent: pull anything available, reroll only if the expected score of the next roll is higher than the current score

def score_naive(dice):
    """
    Input: a roll of 1-6 dice
    Strategy: pull anything available, return the score gained and number of remaining dice
    """
    counts = [0] * 6
    for die in dice: counts[die - 1] += 1
    score = 0
    if counts[0] >= 3:
        score += 1000 + (counts[0] - 3) * 100
        counts[0] = 0
    for i in range(1, 6):
        count = counts[i]
        if count >= 3:
            score += (i + 1) * 100 + (count - 3) * 100
            counts[i] = 0
    score += counts[0] * 100
    counts[0] = 0
    score += counts[4] * 50
    counts[4] = 0
    remaining_dice = sum(counts)
    return score, remaining_dice

# Dynammic programming approach: 
# exp_turn(1) = exp_scores(1)
# exp_turn(2) = roll_score(2) + exp_turn(1 or 0)
# exp_turn(3) = roll_score(3) + exp_turn(2, 1, or 0)

exp_turns = [0] * 7
for n in range(0, 7):
    dice = [list(range(1, 7)) for _ in range(n)]
    p_roll = 6 ** -n
    exp_turn = 0
    for roll in product(*dice):
        # Naive strategy
        roll_exp_score, remaining_dice = score_naive(roll)
        reroll_exp_score = exp_turns[remaining_dice]
        if reroll_exp_score > roll_exp_score:
            roll_exp_score = (roll_exp_score + reroll_exp_score) / 2  # TODO: Fix this, probability of skunk is not 1/2
        exp_turn += roll_exp_score * p_roll
    print(f"{n} dice, E[Turn] = {round(exp_turn, 2)}")
    exp_turns[n] = exp_turn

0 dice, E[Turn] = 0
1 dice, E[Turn] = 25.0
2 dice, E[Turn] = 50.0
3 dice, E[Turn] = 86.81
4 dice, E[Turn] = 145.07
5 dice, E[Turn] = 226.26
6 dice, E[Turn] = 326.2
